# Introduction to Snowpark pandas
The Snowpark pandas API is an API built on top of Snowpark that allows you to run your pandas code directly on your data in Snowflake. Built to replicate the functionality of pandas - including its data isolation and consistency guarantees - the Snowpark pandas API enables you to scale up your pandas-native pipelines simply by replacing an import statement.

In today's demo, we'll be taking a look at how you can get started with the API, as well as comparing its performance with native pandas.

## Importing Snowpark pandas
Much like Snowpark, Snowpark pandas requires an active `Session` object to connect to your data in Snowflake. In the next cell, we'll be initializing a Session object, and importing both Snowpark pandas and vanilla pandas, as `pd` and `native_pd` respectively.

In [1]:
import pandas as native_pd
import modin.pandas as pd
# Import the Snowpark pandas plugin for modin
import snowflake.snowpark.modin.plugin
from snowflake.snowpark.session import Session
# Create a Snowpark session with a default connection.
session = Session.builder.create()

## Getting Started - Reading Data from Snowflake
Today, we'll be analyzing some COVID-19 Data from Snowflake's Marketplace. The data is available courtesy of Starschema, and can be found [here](https://app.snowflake.com/marketplace/listing/GZSNZ7F5UH/starschema-covid-19-epidemiological-data). Let's start by reading the `CDC_TESTING` table into a DataFrame!

In [2]:
DATA_PATH = "COVID19_EPIDEMIOLOGICAL_DATA_1.PUBLIC"

In [3]:
# Snowpark pandas
from time import perf_counter
start = perf_counter()
snow_df = pd.read_snowflake(f"{DATA_PATH}.CDC_TESTING")
end = perf_counter()
print(f"Snowpark pandas took {end - start} seconds to read the data!")

Snowpark pandas took 1.8032602090388536 seconds to read the data!


In [4]:
# Vanilla pandas
from IPython import display
start = perf_counter()

# Create a cursor object.
cur = session.connection.cursor()

# Execute a statement that will generate a result set.
sql = f"SELECT * FROM {DATA_PATH}.CDC_TESTING"
cur.execute(sql)

# Fetch the result set from the cursor and deliver it as the pandas DataFrame.
native_df = cur.fetch_pandas_all()
end = perf_counter()
print(f"Vanilla pandas took {end - start} seconds to read the data!")

Vanilla pandas took 0.6728844169992954 seconds to read the data!


In [5]:
native_df

,ISO3166_1,ISO3166_2,DATE,POSITIVE,NEGATIVE,INCONCLUSIVE
0,US,AK,2020-03-01,NaN,4.0,NaN
1,US,AK,2020-03-02,NaN,0.0,NaN
2,US,AK,2020-03-03,NaN,2.0,NaN
3,US,AK,2020-03-04,NaN,12.0,NaN
4,US,AK,2020-03-05,NaN,9.0,NaN
...,...,...,...,...,...,...
50102,US,WY,2022-08-21,33.0,93.0,5.0
50103,US,WY,2022-08-22,52.0,422.0,1.0
50104,US,WY,2022-08-23,79.0,439.0,0.0
50105,US,WY,2022-08-24,67.0,617.0,8.0


In [6]:
snow_df

,ISO3166_1,ISO3166_2,DATE,POSITIVE,NEGATIVE,INCONCLUSIVE
0,US,AK,2020-03-01,NaN,4,NaN
1,US,AK,2020-03-02,NaN,0,NaN
2,US,AK,2020-03-03,NaN,2,NaN
3,US,AK,2020-03-04,NaN,12,NaN
4,US,AK,2020-03-05,NaN,9,NaN
...,...,...,...,...,...,...
50102,US,WY,2022-08-21,33.0,93,5.0
50103,US,WY,2022-08-22,52.0,422,1.0
50104,US,WY,2022-08-23,79.0,439,0.0
50105,US,WY,2022-08-24,67.0,617,8.0


## Examining the Raw Data
Above, we see that both of the DataFrame's look the same - as expected. Examining the data though, there's a couple of things that pop out. The first, is that the first two columns have fairly nondescript names - so it may be useful to rename them.

In [7]:
start = perf_counter()
snow_df = snow_df.rename(columns={"ISO3166_1": "Nation", "ISO3166_2": "State"})
repr(snow_df)
end = perf_counter()
print(f"\nRename Took {end-start} seconds in Snowpark pandas!")
snow_df


Rename Took 0.2180187499616295 seconds in Snowpark pandas!


,Nation,State,DATE,POSITIVE,NEGATIVE,INCONCLUSIVE
0,US,AK,2020-03-01,NaN,4,NaN
1,US,AK,2020-03-02,NaN,0,NaN
2,US,AK,2020-03-03,NaN,2,NaN
3,US,AK,2020-03-04,NaN,12,NaN
4,US,AK,2020-03-05,NaN,9,NaN
...,...,...,...,...,...,...
50102,US,WY,2022-08-21,33.0,93,5.0
50103,US,WY,2022-08-22,52.0,422,1.0
50104,US,WY,2022-08-23,79.0,439,0.0
50105,US,WY,2022-08-24,67.0,617,8.0


In [8]:
start = perf_counter()
native_df = native_df.rename(columns={"ISO3166_1": "Nation", "ISO3166_2": "State"})
repr(native_df)
end = perf_counter()
print(f"Rename Took {end-start} seconds in vanilla pandas!")
native_df

Rename Took 0.004496291978284717 seconds in vanilla pandas!


,Nation,State,DATE,POSITIVE,NEGATIVE,INCONCLUSIVE
0,US,AK,2020-03-01,NaN,4.0,NaN
1,US,AK,2020-03-02,NaN,0.0,NaN
2,US,AK,2020-03-03,NaN,2.0,NaN
3,US,AK,2020-03-04,NaN,12.0,NaN
4,US,AK,2020-03-05,NaN,9.0,NaN
...,...,...,...,...,...,...
50102,US,WY,2022-08-21,33.0,93.0,5.0
50103,US,WY,2022-08-22,52.0,422.0,1.0
50104,US,WY,2022-08-23,79.0,439.0,0.0
50105,US,WY,2022-08-24,67.0,617.0,8.0


We also notice that it seems like every enter has the same value for `Nation` - we can confirm that that is the case, and then drop the column, since we already know that this data is for the U.S.!

In [9]:
start = perf_counter()
assert (snow_df.Nation == "US").all()
snow_df = snow_df.drop(columns="Nation")
repr(snow_df)
end = perf_counter()
print(f"\nCheck for all values == 'US' and drop Nation column took {end-start} seconds in Snowpark pandas!")
snow_df


Check for all values == 'US' and drop Nation column took 0.8058736670063809 seconds in Snowpark pandas!


,State,DATE,POSITIVE,NEGATIVE,INCONCLUSIVE
0,AK,2020-03-01,NaN,4,NaN
1,AK,2020-03-02,NaN,0,NaN
2,AK,2020-03-03,NaN,2,NaN
3,AK,2020-03-04,NaN,12,NaN
4,AK,2020-03-05,NaN,9,NaN
...,...,...,...,...,...
50102,WY,2022-08-21,33.0,93,5.0
50103,WY,2022-08-22,52.0,422,1.0
50104,WY,2022-08-23,79.0,439,0.0
50105,WY,2022-08-24,67.0,617,8.0


In [10]:
start = perf_counter()
assert (native_df.Nation == "US").all()
native_df = native_df.drop(columns="Nation")
repr(native_df)
end = perf_counter()
print(f"Check for all values == 'US' and drop Nation column took {end-start} seconds in vanilla pandas!")
native_df

Check for all values == 'US' and drop Nation column took 0.009369750041514635 seconds in vanilla pandas!


,State,DATE,POSITIVE,NEGATIVE,INCONCLUSIVE
0,AK,2020-03-01,NaN,4.0,NaN
1,AK,2020-03-02,NaN,0.0,NaN
2,AK,2020-03-03,NaN,2.0,NaN
3,AK,2020-03-04,NaN,12.0,NaN
4,AK,2020-03-05,NaN,9.0,NaN
...,...,...,...,...,...
50102,WY,2022-08-21,33.0,93.0,5.0
50103,WY,2022-08-22,52.0,422.0,1.0
50104,WY,2022-08-23,79.0,439.0,0.0
50105,WY,2022-08-24,67.0,617.0,8.0


# Reshaping the Data
Now that we've done some preliminary EDA, we can move on to analyzing the data. The data is well formatted now if we were to attempt a groupby, for example, but let's say we wanted to compare the differences in testing results across states for each day - it would be more useful if we could pivot the data so that the columns are states, so we can take a look at the cases per state per day more easily. To accomplish this, we can use the `pivot_table` API!

In [11]:
start = perf_counter()
snow_df = snow_df.pivot_table(columns=["State"], values = ["POSITIVE", "NEGATIVE", "INCONCLUSIVE"], index="DATE")
repr(snow_df)
end = perf_counter()
print(f"Pivoting the table took {end-start} seconds in Snowpark pandas!")
snow_df

Pivoting the table took 5.211480959085748 seconds in Snowpark pandas!


POSITIVE                                                          \
State            AK      AL     AR     AZ    CA     CO     CT     DC     DE   
DATE                                                                          
2020-03-01      NaN    16.0    NaN    NaN  12.0    NaN    NaN    NaN    NaN   
2020-03-02      NaN     6.0    NaN   11.0  16.0    NaN    NaN    NaN    NaN   
2020-03-03      NaN     9.0    NaN    0.0  10.0    NaN    NaN    NaN    NaN   
2020-03-04      NaN     2.0    NaN    0.0   6.0    NaN    NaN    NaN    NaN   
2020-03-05      NaN     6.0    NaN    2.0  19.0    NaN    NaN    NaN    NaN   
...             ...     ...    ...    ...   ...    ...    ...    ...    ...   
2022-08-22    275.0  1395.0  538.0  789.0   NaN  683.0  752.0  121.0  250.0   
2022-08-23    174.0  1665.0  559.0  990.0   NaN  947.0  776.0  241.0  341.0   
2022-08-24     36.0  1056.0  471.0  808.0   NaN  972.0  745.0  175.0  207.0   
2022-08-25      NaN     NaN  457.0   93.0   NaN  910.0    NaN  119.0   43.0   
2022-08-26      NaN     NaN    NaN    NaN   NaN  116.0    NaN   64.0    NaN   

                    ... INCONCLUSIVE                                           \
State           FL  ...           TN    TX    UT    VA   VI    VT    WA    WI   
DATE                ...                                                         
2020-03-01     NaN  ...          NaN   NaN   NaN   NaN  NaN   NaN   1.0   NaN   
2020-03-02     6.0  ...          NaN   NaN   NaN   NaN  NaN   NaN   0.0   NaN   
2020-03-03     0.0  ...          NaN   NaN   NaN   NaN  NaN   NaN   0.0   NaN   
2020-03-04    16.0  ...          NaN   NaN   NaN   NaN  NaN   NaN   0.0   NaN   
2020-03-05    20.0  ...          NaN   NaN   NaN   NaN  NaN   NaN   0.0   NaN   
...            ...  ...          ...   ...   ...   ...  ...   ...   ...   ...   
2022-08-22  6869.0  ...          5.0  42.0   8.0  18.0  0.0   0.0  30.0  22.0   
2022-08-23  5617.0  ...          6.0  42.0  14.0  18.0  0.0  12.0  16.0  18.0   
2022-08-24  3645.0  ...          4.0  13.0   7.0   8.0  0.0   9.0   0.0  13.0   
2022-08-25   448.0  ...          NaN   0.0  16.0   4.0  0.0   7.0   NaN  11.0   
2022-08-26     NaN  ...          NaN   NaN   NaN   NaN  NaN   NaN   NaN   1.0   

                       
State         WV   WY  
DATE                   
2020-03-01   NaN  NaN  
2020-03-02   NaN  NaN  
2020-03-03   NaN  NaN  
2020-03-04   NaN  NaN  
2020-03-05   NaN  NaN  
...          ...  ...  
2022-08-22   3.0  1.0  
2022-08-23   3.0  0.0  
2022-08-24  10.0  8.0  
2022-08-25  12.0  4.0  
2022-08-26   NaN  NaN  

[909 rows x 168 columns]

In [12]:
start = perf_counter()
native_df = native_df.pivot_table(columns=["State"], values = ["POSITIVE", "NEGATIVE", "INCONCLUSIVE"], index="DATE")
repr(native_df)
end = perf_counter()
print(f"Pivoting the table took {end-start} seconds in vanilla pandas!")
native_df

Pivoting the table took 0.014328707940876484 seconds in vanilla pandas!


INCONCLUSIVE                                                    \
State                AK   AL   AR    AZ  CA    CO    CT    DC   DE     FL   
DATE                                                                        
2020-03-01          NaN  NaN  NaN   NaN NaN   NaN   NaN   NaN  NaN    NaN   
2020-03-02          NaN  NaN  NaN   NaN NaN   NaN   NaN   NaN  NaN    NaN   
2020-03-03          NaN  NaN  NaN   NaN NaN   NaN   NaN   NaN  NaN    NaN   
2020-03-04          NaN  NaN  NaN   NaN NaN   NaN   NaN   NaN  NaN    NaN   
2020-03-05          NaN  NaN  NaN   NaN NaN   NaN   NaN   NaN  NaN    NaN   
...                 ...  ...  ...   ...  ..   ...   ...   ...  ...    ...   
2022-08-22          3.0  7.0  3.0  17.0 NaN  14.0   7.0   6.0  0.0  391.0   
2022-08-23          0.0  3.0  4.0   6.0 NaN  20.0  16.0  10.0  1.0  304.0   
2022-08-24          0.0  9.0  3.0   6.0 NaN  20.0  15.0   6.0  3.0  123.0   
2022-08-25          NaN  NaN  8.0   2.0 NaN  25.0   NaN   7.0  0.0    3.0   
2022-08-26          NaN  NaN  NaN   NaN NaN   2.0   NaN   5.0  NaN    NaN   

            ... POSITIVE                                                      \
State       ...       TN      TX     UT      VA    VI     VT      WA      WI   
DATE        ...                                                                
2020-03-01  ...      NaN     NaN    NaN     NaN   NaN    NaN    28.0     NaN   
2020-03-02  ...      NaN     NaN    NaN     NaN   NaN    NaN    18.0     NaN   
2020-03-03  ...      1.0     NaN    NaN     1.0   NaN    NaN     6.0     NaN   
2020-03-04  ...      0.0     NaN    NaN     0.0   NaN    NaN    21.0     NaN   
2020-03-05  ...      2.0     1.0    NaN     0.0   NaN    NaN    10.0     NaN   
...         ...      ...     ...    ...     ...   ...    ...     ...     ...   
2022-08-22  ...   1589.0  3491.0  377.0  2002.0   7.0   56.0  1006.0  1328.0   
2022-08-23  ...   2069.0  4695.0  568.0  2075.0  19.0  117.0   389.0  1504.0   
2022-08-24  ...    556.0  2500.0  475.0  1403.0   6.0   94.0     0.0  1531.0   
2022-08-25  ...      NaN    58.0  350.0   232.0  17.0   42.0     NaN  1251.0   
2022-08-26  ...      NaN     NaN    NaN     NaN   NaN    NaN     NaN   253.0   

                         
State          WV    WY  
DATE                     
2020-03-01    NaN   NaN  
2020-03-02    NaN   NaN  
2020-03-03    NaN   NaN  
2020-03-04    NaN   NaN  
2020-03-05    NaN   NaN  
...           ...   ...  
2022-08-22  545.0  52.0  
2022-08-23  715.0  79.0  
2022-08-24  762.0  67.0  
2022-08-25  555.0  25.0  
2022-08-26    NaN   NaN  

[909 rows x 167 columns]

For convenience, let's drop the `NEGATIVE` and `INCONCLUSIVE` values, and focus more closely on the `POSITIVE` test results!

In [13]:
start = perf_counter()
snow_df = snow_df.drop(columns=["NEGATIVE", "INCONCLUSIVE"], level=0)
repr(snow_df)
end = perf_counter()
print(f"Dropping NEGATIVE and INCONCLUSIVE values took {end-start} seconds in Snowpark pandas!")
snow_df

Dropping NEGATIVE and INCONCLUSIVE values took 2.3781988330883905 seconds in Snowpark pandas!


POSITIVE                                                          \
State            AK      AL     AR     AZ    CA     CO     CT     DC     DE   
DATE                                                                          
2020-03-01      NaN    16.0    NaN    NaN  12.0    NaN    NaN    NaN    NaN   
2020-03-02      NaN     6.0    NaN   11.0  16.0    NaN    NaN    NaN    NaN   
2020-03-03      NaN     9.0    NaN    0.0  10.0    NaN    NaN    NaN    NaN   
2020-03-04      NaN     2.0    NaN    0.0   6.0    NaN    NaN    NaN    NaN   
2020-03-05      NaN     6.0    NaN    2.0  19.0    NaN    NaN    NaN    NaN   
...             ...     ...    ...    ...   ...    ...    ...    ...    ...   
2022-08-22    275.0  1395.0  538.0  789.0   NaN  683.0  752.0  121.0  250.0   
2022-08-23    174.0  1665.0  559.0  990.0   NaN  947.0  776.0  241.0  341.0   
2022-08-24     36.0  1056.0  471.0  808.0   NaN  972.0  745.0  175.0  207.0   
2022-08-25      NaN     NaN  457.0   93.0   NaN  910.0    NaN  119.0   43.0   
2022-08-26      NaN     NaN    NaN    NaN   NaN  116.0    NaN   64.0    NaN   

                    ...                                                      \
State           FL  ...      TN      TX     UT      VA    VI     VT      WA   
DATE                ...                                                       
2020-03-01     NaN  ...     NaN     NaN    NaN     NaN   NaN    NaN    28.0   
2020-03-02     6.0  ...     NaN     NaN    NaN     NaN   NaN    NaN    18.0   
2020-03-03     0.0  ...     1.0     NaN    NaN     1.0   NaN    NaN     6.0   
2020-03-04    16.0  ...     0.0     NaN    NaN     0.0   NaN    NaN    21.0   
2020-03-05    20.0  ...     2.0     1.0    NaN     0.0   NaN    NaN    10.0   
...            ...  ...     ...     ...    ...     ...   ...    ...     ...   
2022-08-22  6869.0  ...  1589.0  3491.0  377.0  2002.0   7.0   56.0  1006.0   
2022-08-23  5617.0  ...  2069.0  4695.0  568.0  2075.0  19.0  117.0   389.0   
2022-08-24  3645.0  ...   556.0  2500.0  475.0  1403.0   6.0   94.0     0.0   
2022-08-25   448.0  ...     NaN    58.0  350.0   232.0  17.0   42.0     NaN   
2022-08-26     NaN  ...     NaN     NaN    NaN     NaN   NaN    NaN     NaN   

                                 
State           WI     WV    WY  
DATE                             
2020-03-01     NaN    NaN   NaN  
2020-03-02     NaN    NaN   NaN  
2020-03-03     NaN    NaN   NaN  
2020-03-04     NaN    NaN   NaN  
2020-03-05     NaN    NaN   NaN  
...            ...    ...   ...  
2022-08-22  1328.0  545.0  52.0  
2022-08-23  1504.0  715.0  79.0  
2022-08-24  1531.0  762.0  67.0  
2022-08-25  1251.0  555.0  25.0  
2022-08-26   253.0    NaN   NaN  

[909 rows x 56 columns]

In [14]:
start = perf_counter()
native_df = native_df.drop(columns=["NEGATIVE", "INCONCLUSIVE"], level=0)
repr(native_df)
end = perf_counter()
print(f"Dropping NEGATIVE and INCONCLUSIVE values took {end-start} seconds in vanilla pandas!")
native_df

Dropping NEGATIVE and INCONCLUSIVE values took 0.006450708024203777 seconds in vanilla pandas!


POSITIVE                                                          \
State            AK      AL     AR     AZ    CA     CO     CT     DC     DE   
DATE                                                                          
2020-03-01      NaN    16.0    NaN    NaN  12.0    NaN    NaN    NaN    NaN   
2020-03-02      NaN     6.0    NaN   11.0  16.0    NaN    NaN    NaN    NaN   
2020-03-03      NaN     9.0    NaN    0.0  10.0    NaN    NaN    NaN    NaN   
2020-03-04      NaN     2.0    NaN    0.0   6.0    NaN    NaN    NaN    NaN   
2020-03-05      NaN     6.0    NaN    2.0  19.0    NaN    NaN    NaN    NaN   
...             ...     ...    ...    ...   ...    ...    ...    ...    ...   
2022-08-22    275.0  1395.0  538.0  789.0   NaN  683.0  752.0  121.0  250.0   
2022-08-23    174.0  1665.0  559.0  990.0   NaN  947.0  776.0  241.0  341.0   
2022-08-24     36.0  1056.0  471.0  808.0   NaN  972.0  745.0  175.0  207.0   
2022-08-25      NaN     NaN  457.0   93.0   NaN  910.0    NaN  119.0   43.0   
2022-08-26      NaN     NaN    NaN    NaN   NaN  116.0    NaN   64.0    NaN   

                    ...                                                      \
State           FL  ...      TN      TX     UT      VA    VI     VT      WA   
DATE                ...                                                       
2020-03-01     NaN  ...     NaN     NaN    NaN     NaN   NaN    NaN    28.0   
2020-03-02     6.0  ...     NaN     NaN    NaN     NaN   NaN    NaN    18.0   
2020-03-03     0.0  ...     1.0     NaN    NaN     1.0   NaN    NaN     6.0   
2020-03-04    16.0  ...     0.0     NaN    NaN     0.0   NaN    NaN    21.0   
2020-03-05    20.0  ...     2.0     1.0    NaN     0.0   NaN    NaN    10.0   
...            ...  ...     ...     ...    ...     ...   ...    ...     ...   
2022-08-22  6869.0  ...  1589.0  3491.0  377.0  2002.0   7.0   56.0  1006.0   
2022-08-23  5617.0  ...  2069.0  4695.0  568.0  2075.0  19.0  117.0   389.0   
2022-08-24  3645.0  ...   556.0  2500.0  475.0  1403.0   6.0   94.0     0.0   
2022-08-25   448.0  ...     NaN    58.0  350.0   232.0  17.0   42.0     NaN   
2022-08-26     NaN  ...     NaN     NaN    NaN     NaN   NaN    NaN     NaN   

                                 
State           WI     WV    WY  
DATE                             
2020-03-01     NaN    NaN   NaN  
2020-03-02     NaN    NaN   NaN  
2020-03-03     NaN    NaN   NaN  
2020-03-04     NaN    NaN   NaN  
2020-03-05     NaN    NaN   NaN  
...            ...    ...   ...  
2022-08-22  1328.0  545.0  52.0  
2022-08-23  1504.0  715.0  79.0  
2022-08-24  1531.0  762.0  67.0  
2022-08-25  1251.0  555.0  25.0  
2022-08-26   253.0    NaN   NaN  

[909 rows x 56 columns]

## Transforming the Data
The granularity of the data we have is daily - what if we wanted to take a look at weekly trends? We can use the `resample` API to resample our Index on a weekly basis!

In [15]:
start = perf_counter()
snow_df = snow_df.resample('7D').sum()
repr(snow_df)
end = perf_counter()
print(f"Resampling weekly took {end-start} seconds in Snowpark pandas!")
snow_df

Resampling weekly took 5.5374357079854235 seconds in Snowpark pandas!


POSITIVE                                                     \
State            AK      AL      AR      AZ        CA       CO      CT   
DATE                                                                     
2020-03-01      0.0    47.0     0.0    16.0     136.0      0.0     2.0   
2020-03-08      3.0   107.0     6.0    17.0     588.0     29.0    43.0   
2020-03-15     50.0   370.0    67.0   194.0    1859.0     46.0   581.0   
2020-03-22     89.0  1545.0   189.0  1337.0    6388.0     26.0  3312.0   
2020-03-29    117.0  2122.0   281.0  2187.0   10081.0     29.0  6487.0   
...             ...     ...     ...     ...       ...      ...     ...   
2022-07-24   3108.0  9364.0  4093.0  9305.0  134023.0  10124.0  6187.0   
2022-07-31   2692.0  8978.0  3689.0  9351.0  109371.0   8682.0  6026.0   
2022-08-07   2209.0  8134.0  3444.0  7736.0   92084.0   7291.0  5955.0   
2022-08-14   1729.0  8630.0  3097.0  6490.0   67628.0   5877.0  5012.0   
2022-08-21    612.0  4353.0  2257.0  3194.0    2944.0   4218.0  2826.0   

                                     ...                                     \
State           DC      DE       FL  ...       TN       TX      UT       VA   
DATE                                 ...                                      
2020-03-01     0.0     0.0     65.0  ...      7.0      1.0     0.0      1.0   
2020-03-08    20.0     1.0    558.0  ...     59.0     33.0    68.0     10.0   
2020-03-15   183.0    62.0   5180.0  ...    547.0    807.0   412.0    324.0   
2020-03-22   608.0   224.0  11101.0  ...   1694.0   3488.0  1436.0   1098.0   
2020-03-29  1018.0   911.0  11743.0  ...   3535.0   3881.0  2421.0   2654.0   
...            ...     ...      ...  ...      ...      ...     ...      ...   
2022-07-24  2378.0  2537.0  66998.0  ...   8982.0  71128.0  4969.0  16311.0   
2022-07-31  1611.0  2851.0  56965.0  ...  10176.0  57769.0  4956.0  17654.0   
2022-08-07  1302.0  2413.0  46598.0  ...   9500.0  49303.0  3889.0  15465.0   
2022-08-14  1260.0  2106.0  41094.0  ...  11003.0  43080.0  3090.0  14527.0   
2022-08-21   850.0   989.0  21016.0  ...   5085.0  14949.0  2071.0   7242.0   

                                                          
State         VI     VT       WA       WI      WV     WY  
DATE                                                      
2020-03-01   0.0    0.0    128.0      0.0     0.0    0.0  
2020-03-08   0.0    0.0    250.0      9.0     0.0    0.0  
2020-03-15   0.0   57.0    562.0    439.0    15.0    0.0  
2020-03-22   0.0  189.0   1776.0    898.0   147.0    1.0  
2020-03-29   1.0  380.0   1594.0   1200.0   321.0    0.0  
...          ...    ...      ...      ...     ...    ...  
2022-07-24  70.0  452.0  12354.0  11690.0  4013.0  641.0  
2022-07-31  81.0  524.0  10895.0  10825.0  4246.0  741.0  
2022-08-07  55.0  473.0   9733.0  10146.0  4176.0  674.0  
2022-08-14  61.0  527.0   8244.0   9488.0  3819.0  537.0  
2022-08-21  52.0  360.0   2055.0   6653.0  2973.0  256.0  

[130 rows x 56 columns]

In [16]:
start = perf_counter()
native_df = native_df.resample("7D").sum()
repr(native_df)
end = perf_counter()
print(f"Resampling weekly took {end-start} seconds in vanilla pandas!")
native_df

Resampling weekly took 0.007296917028725147 seconds in vanilla pandas!


POSITIVE                                                     \
State            AK      AL      AR      AZ        CA       CO      CT   
DATE                                                                     
2020-03-01      0.0    47.0     0.0    16.0     136.0      0.0     2.0   
2020-03-08      3.0   107.0     6.0    17.0     588.0     29.0    43.0   
2020-03-15     50.0   370.0    67.0   194.0    1859.0     46.0   581.0   
2020-03-22     89.0  1545.0   189.0  1337.0    6388.0     26.0  3312.0   
2020-03-29    117.0  2122.0   281.0  2187.0   10081.0     29.0  6487.0   
...             ...     ...     ...     ...       ...      ...     ...   
2022-07-24   3108.0  9364.0  4093.0  9305.0  134023.0  10124.0  6187.0   
2022-07-31   2692.0  8978.0  3689.0  9351.0  109371.0   8682.0  6026.0   
2022-08-07   2209.0  8134.0  3444.0  7736.0   92084.0   7291.0  5955.0   
2022-08-14   1729.0  8630.0  3097.0  6490.0   67628.0   5877.0  5012.0   
2022-08-21    612.0  4353.0  2257.0  3194.0    2944.0   4218.0  2826.0   

                                     ...                                     \
State           DC      DE       FL  ...       TN       TX      UT       VA   
DATE                                 ...                                      
2020-03-01     0.0     0.0     65.0  ...      7.0      1.0     0.0      1.0   
2020-03-08    20.0     1.0    558.0  ...     59.0     33.0    68.0     10.0   
2020-03-15   183.0    62.0   5180.0  ...    547.0    807.0   412.0    324.0   
2020-03-22   608.0   224.0  11101.0  ...   1694.0   3488.0  1436.0   1098.0   
2020-03-29  1018.0   911.0  11743.0  ...   3535.0   3881.0  2421.0   2654.0   
...            ...     ...      ...  ...      ...      ...     ...      ...   
2022-07-24  2378.0  2537.0  66998.0  ...   8982.0  71128.0  4969.0  16311.0   
2022-07-31  1611.0  2851.0  56965.0  ...  10176.0  57769.0  4956.0  17654.0   
2022-08-07  1302.0  2413.0  46598.0  ...   9500.0  49303.0  3889.0  15465.0   
2022-08-14  1260.0  2106.0  41094.0  ...  11003.0  43080.0  3090.0  14527.0   
2022-08-21   850.0   989.0  21016.0  ...   5085.0  14949.0  2071.0   7242.0   

                                                          
State         VI     VT       WA       WI      WV     WY  
DATE                                                      
2020-03-01   0.0    0.0    128.0      0.0     0.0    0.0  
2020-03-08   0.0    0.0    250.0      9.0     0.0    0.0  
2020-03-15   0.0   57.0    562.0    439.0    15.0    0.0  
2020-03-22   0.0  189.0   1776.0    898.0   147.0    1.0  
2020-03-29   1.0  380.0   1594.0   1200.0   321.0    0.0  
...          ...    ...      ...      ...     ...    ...  
2022-07-24  70.0  452.0  12354.0  11690.0  4013.0  641.0  
2022-07-31  81.0  524.0  10895.0  10825.0  4246.0  741.0  
2022-08-07  55.0  473.0   9733.0  10146.0  4176.0  674.0  
2022-08-14  61.0  527.0   8244.0   9488.0  3819.0  537.0  
2022-08-21  52.0  360.0   2055.0   6653.0  2973.0  256.0  

[130 rows x 56 columns]

Let's say we wanted to see the week-over-week trend - we can accomplish this using the `diff` API!

In [17]:
start = perf_counter()
snow_df_diff = snow_df.diff()
repr(snow_df_diff)
end = perf_counter()
print(f"Difference week-over-week took {end-start} seconds in Snowpark pandas!")
snow_df_diff

Difference week-over-week took 3.623762540984899 seconds in Snowpark pandas!


POSITIVE                                                         \
State            AK      AL     AR      AZ       CA      CO      CT     DC   
DATE                                                                         
2020-03-01      NaN     NaN    NaN     NaN      NaN     NaN     NaN    NaN   
2020-03-08      3.0    60.0    6.0     1.0    452.0    29.0    41.0   20.0   
2020-03-15     47.0   263.0   61.0   177.0   1271.0    17.0   538.0  163.0   
2020-03-22     39.0  1175.0  122.0  1143.0   4529.0   -20.0  2731.0  425.0   
2020-03-29     28.0   577.0   92.0   850.0   3693.0     3.0  3175.0  410.0   
...             ...     ...    ...     ...      ...     ...     ...    ...   
2022-07-24    -31.0  -207.0 -731.0 -2447.0 -20206.0 -1693.0  -445.0 -192.0   
2022-07-31   -416.0  -386.0 -404.0    46.0 -24652.0 -1442.0  -161.0 -767.0   
2022-08-07   -483.0  -844.0 -245.0 -1615.0 -17287.0 -1391.0   -71.0 -309.0   
2022-08-14   -480.0   496.0 -347.0 -1246.0 -24456.0 -1414.0  -943.0  -42.0   
2022-08-21  -1117.0 -4277.0 -840.0 -3296.0 -64684.0 -1659.0 -2186.0 -410.0   

                             ...                                         \
State           DE       FL  ...      TN       TX      UT      VA    VI   
DATE                         ...                                          
2020-03-01     NaN      NaN  ...     NaN      NaN     NaN     NaN   NaN   
2020-03-08     1.0    493.0  ...    52.0     32.0    68.0     9.0   0.0   
2020-03-15    61.0   4622.0  ...   488.0    774.0   344.0   314.0   0.0   
2020-03-22   162.0   5921.0  ...  1147.0   2681.0  1024.0   774.0   0.0   
2020-03-29   687.0    642.0  ...  1841.0    393.0   985.0  1556.0   1.0   
...            ...      ...  ...     ...      ...     ...     ...   ...   
2022-07-24   -72.0  -4448.0  ... -1936.0  -5323.0  -870.0   536.0 -16.0   
2022-07-31   314.0 -10033.0  ...  1194.0 -13359.0   -13.0  1343.0  11.0   
2022-08-07  -438.0 -10367.0  ...  -676.0  -8466.0 -1067.0 -2189.0 -26.0   
2022-08-14  -307.0  -5504.0  ...  1503.0  -6223.0  -799.0  -938.0   6.0   
2022-08-21 -1117.0 -20078.0  ... -5918.0 -28131.0 -1019.0 -7285.0  -9.0   

                                                 
State          VT      WA      WI     WV     WY  
DATE                                             
2020-03-01    NaN     NaN     NaN    NaN    NaN  
2020-03-08    0.0   122.0     9.0    0.0    0.0  
2020-03-15   57.0   312.0   430.0   15.0    0.0  
2020-03-22  132.0  1214.0   459.0  132.0    1.0  
2020-03-29  191.0  -182.0   302.0  174.0   -1.0  
...           ...     ...     ...    ...    ...  
2022-07-24  -23.0 -3126.0   -18.0 -110.0 -240.0  
2022-07-31   72.0 -1459.0  -865.0  233.0  100.0  
2022-08-07  -51.0 -1162.0  -679.0  -70.0  -67.0  
2022-08-14   54.0 -1489.0  -658.0 -357.0 -137.0  
2022-08-21 -167.0 -6189.0 -2835.0 -846.0 -281.0  

[130 rows x 56 columns]

In [18]:
start = perf_counter()
native_df_diff = native_df.diff()
repr(native_df_diff)
end = perf_counter()
print(f"Difference week-over-week took {end-start} seconds in vanilla pandas!")
native_df_diff

Difference week-over-week took 0.005066458019427955 seconds in vanilla pandas!


POSITIVE                                                         \
State            AK      AL     AR      AZ       CA      CO      CT     DC   
DATE                                                                         
2020-03-01      NaN     NaN    NaN     NaN      NaN     NaN     NaN    NaN   
2020-03-08      3.0    60.0    6.0     1.0    452.0    29.0    41.0   20.0   
2020-03-15     47.0   263.0   61.0   177.0   1271.0    17.0   538.0  163.0   
2020-03-22     39.0  1175.0  122.0  1143.0   4529.0   -20.0  2731.0  425.0   
2020-03-29     28.0   577.0   92.0   850.0   3693.0     3.0  3175.0  410.0   
...             ...     ...    ...     ...      ...     ...     ...    ...   
2022-07-24    -31.0  -207.0 -731.0 -2447.0 -20206.0 -1693.0  -445.0 -192.0   
2022-07-31   -416.0  -386.0 -404.0    46.0 -24652.0 -1442.0  -161.0 -767.0   
2022-08-07   -483.0  -844.0 -245.0 -1615.0 -17287.0 -1391.0   -71.0 -309.0   
2022-08-14   -480.0   496.0 -347.0 -1246.0 -24456.0 -1414.0  -943.0  -42.0   
2022-08-21  -1117.0 -4277.0 -840.0 -3296.0 -64684.0 -1659.0 -2186.0 -410.0   

                             ...                                         \
State           DE       FL  ...      TN       TX      UT      VA    VI   
DATE                         ...                                          
2020-03-01     NaN      NaN  ...     NaN      NaN     NaN     NaN   NaN   
2020-03-08     1.0    493.0  ...    52.0     32.0    68.0     9.0   0.0   
2020-03-15    61.0   4622.0  ...   488.0    774.0   344.0   314.0   0.0   
2020-03-22   162.0   5921.0  ...  1147.0   2681.0  1024.0   774.0   0.0   
2020-03-29   687.0    642.0  ...  1841.0    393.0   985.0  1556.0   1.0   
...            ...      ...  ...     ...      ...     ...     ...   ...   
2022-07-24   -72.0  -4448.0  ... -1936.0  -5323.0  -870.0   536.0 -16.0   
2022-07-31   314.0 -10033.0  ...  1194.0 -13359.0   -13.0  1343.0  11.0   
2022-08-07  -438.0 -10367.0  ...  -676.0  -8466.0 -1067.0 -2189.0 -26.0   
2022-08-14  -307.0  -5504.0  ...  1503.0  -6223.0  -799.0  -938.0   6.0   
2022-08-21 -1117.0 -20078.0  ... -5918.0 -28131.0 -1019.0 -7285.0  -9.0   

                                                 
State          VT      WA      WI     WV     WY  
DATE                                             
2020-03-01    NaN     NaN     NaN    NaN    NaN  
2020-03-08    0.0   122.0     9.0    0.0    0.0  
2020-03-15   57.0   312.0   430.0   15.0    0.0  
2020-03-22  132.0  1214.0   459.0  132.0    1.0  
2020-03-29  191.0  -182.0   302.0  174.0   -1.0  
...           ...     ...     ...    ...    ...  
2022-07-24  -23.0 -3126.0   -18.0 -110.0 -240.0  
2022-07-31   72.0 -1459.0  -865.0  233.0  100.0  
2022-08-07  -51.0 -1162.0  -679.0  -70.0  -67.0  
2022-08-14   54.0 -1489.0  -658.0 -357.0 -137.0  
2022-08-21 -167.0 -6189.0 -2835.0 -846.0 -281.0  

[130 rows x 56 columns]

It seems that most of the states began to have rapid declines in new COVID cases per week towards the end of our dataset, which makes intuitive sense. Let's confirm this though, by taking a look at the percentage of cases per week that are new!

In [19]:
import numpy as np
start = perf_counter()
snow_df_percentage = (snow_df_diff / snow_df.replace(0, np.NaN))*100
repr(snow_df_percentage)
end = perf_counter()
print(f"Percentage of new cases week-over-week took {end-start} seconds in Snowpark pandas!")
snow_df_percentage

Percentage of new cases week-over-week took 4.27760200004559 seconds in Snowpark pandas!


POSITIVE                                                  \
State               AK         AL          AR          AZ           CA   
DATE                                                                     
2020-03-01         NaN        NaN         NaN         NaN          NaN   
2020-03-08  100.000000  56.074766  100.000000    5.882353    76.870748   
2020-03-15   94.000000  71.081081   91.044776   91.237113    68.370091   
2020-03-22   43.820225  76.051780   64.550265   85.489903    70.898560   
2020-03-29   23.931624  27.191329   32.740214   38.866027    36.633271   
...                ...        ...         ...         ...          ...   
2022-07-24   -0.997426  -2.210594  -17.859761  -26.297689   -15.076517   
2022-07-31  -15.453195  -4.299399  -10.951477    0.491926   -22.539796   
2022-08-07  -21.865097 -10.376199   -7.113821  -20.876422   -18.773077   
2022-08-14  -27.761712   5.747393  -11.204391  -19.198767   -36.162536   
2022-08-21 -182.516340 -98.254078  -37.217545 -103.193488 -2197.146739   

                                                                      ...  \
State               CO         CT          DC          DE         FL  ...   
DATE                                                                  ...   
2020-03-01         NaN        NaN         NaN         NaN        NaN  ...   
2020-03-08  100.000000  95.348837  100.000000  100.000000  88.351254  ...   
2020-03-15   36.956522  92.598967   89.071038   98.387097  89.227799  ...   
2020-03-22  -76.923077  82.457729   69.901316   72.321429  53.337537  ...   
2020-03-29   10.344828  48.944042   40.275049   75.411636   5.467087  ...   
...                ...        ...         ...         ...        ...  ...   
2022-07-24  -16.722639  -7.192500   -8.074012   -2.837998  -6.639004  ...   
2022-07-31  -16.609076  -2.671756  -47.610180   11.013679 -17.612569  ...   
2022-08-07  -19.078316  -1.192275  -23.732719  -18.151678 -22.247736  ...   
2022-08-14  -24.059895 -18.814844   -3.333333  -14.577398 -13.393683  ...   
2022-08-21  -39.331437 -77.353149  -48.235294 -112.942366 -95.536734  ...   

                                                                        \
State               TN          TX          UT          VA          VI   
DATE                                                                     
2020-03-01         NaN         NaN         NaN         NaN         NaN   
2020-03-08   88.135593   96.969697  100.000000   90.000000         NaN   
2020-03-15   89.213894   95.910781   83.495146   96.913580         NaN   
2020-03-22   67.709563   76.863532   71.309192   70.491803         NaN   
2020-03-29   52.079208   10.126256   40.685667   58.628485  100.000000   
...                ...         ...         ...         ...         ...   
2022-07-24  -21.554220   -7.483691  -17.508553    3.286126  -22.857143   
2022-07-31   11.733491  -23.124859   -0.262308    7.607341   13.580247   
2022-08-07   -7.115789  -17.171369  -27.436359  -14.154543  -47.272727   
2022-08-14   13.659911  -14.445218  -25.857605   -6.456942    9.836066   
2022-08-21 -116.381514 -188.179811  -49.203283 -100.593759  -17.307692   

                                                                        
State               VT          WA          WI          WV          WY  
DATE                                                                    
2020-03-01         NaN         NaN         NaN         NaN         NaN  
2020-03-08         NaN   48.800000  100.000000         NaN         NaN  
2020-03-15  100.000000   55.516014   97.949886  100.000000         NaN  
2020-03-22   69.841270   68.355856   51.113586   89.795918  100.000000  
2020-03-29   50.263158  -11.417817   25.166667   54.205607         NaN  
...                ...         ...         ...         ...         ...  
2022-07-24   -5.088496  -25.303545   -0.153978   -2.741091  -37.441498  
2022-07-31   13.740458  -13.391464   -7.990762    5.487518   13.495277  
2022-08-07  -10.782241  -11.938765   -6.692293   -1.676245

In [20]:
start = perf_counter()
native_df_percentage = (native_df_diff / native_df)*100
repr(native_df_percentage)
end = perf_counter()
print(f"Difference week-over-week took {end-start} seconds in vanilla pandas!")
native_df_percentage

Difference week-over-week took 0.0038729580119252205 seconds in vanilla pandas!


POSITIVE                                                  \
State               AK         AL          AR          AZ           CA   
DATE                                                                     
2020-03-01         NaN        NaN         NaN         NaN          NaN   
2020-03-08  100.000000  56.074766  100.000000    5.882353    76.870748   
2020-03-15   94.000000  71.081081   91.044776   91.237113    68.370091   
2020-03-22   43.820225  76.051780   64.550265   85.489903    70.898560   
2020-03-29   23.931624  27.191329   32.740214   38.866027    36.633271   
...                ...        ...         ...         ...          ...   
2022-07-24   -0.997426  -2.210594  -17.859761  -26.297689   -15.076517   
2022-07-31  -15.453195  -4.299399  -10.951477    0.491926   -22.539796   
2022-08-07  -21.865097 -10.376199   -7.113821  -20.876422   -18.773077   
2022-08-14  -27.761712   5.747393  -11.204391  -19.198767   -36.162536   
2022-08-21 -182.516340 -98.254078  -37.217545 -103.193488 -2197.146739   

                                                                      ...  \
State               CO         CT          DC          DE         FL  ...   
DATE                                                                  ...   
2020-03-01         NaN        NaN         NaN         NaN        NaN  ...   
2020-03-08  100.000000  95.348837  100.000000  100.000000  88.351254  ...   
2020-03-15   36.956522  92.598967   89.071038   98.387097  89.227799  ...   
2020-03-22  -76.923077  82.457729   69.901316   72.321429  53.337537  ...   
2020-03-29   10.344828  48.944042   40.275049   75.411636   5.467087  ...   
...                ...        ...         ...         ...        ...  ...   
2022-07-24  -16.722639  -7.192500   -8.074012   -2.837998  -6.639004  ...   
2022-07-31  -16.609076  -2.671756  -47.610180   11.013679 -17.612569  ...   
2022-08-07  -19.078316  -1.192275  -23.732719  -18.151678 -22.247736  ...   
2022-08-14  -24.059895 -18.814844   -3.333333  -14.577398 -13.393683  ...   
2022-08-21  -39.331437 -77.353149  -48.235294 -112.942366 -95.536734  ...   

                                                                        \
State               TN          TX          UT          VA          VI   
DATE                                                                     
2020-03-01         NaN         NaN         NaN         NaN         NaN   
2020-03-08   88.135593   96.969697  100.000000   90.000000         NaN   
2020-03-15   89.213894   95.910781   83.495146   96.913580         NaN   
2020-03-22   67.709563   76.863532   71.309192   70.491803         NaN   
2020-03-29   52.079208   10.126256   40.685667   58.628485  100.000000   
...                ...         ...         ...         ...         ...   
2022-07-24  -21.554220   -7.483691  -17.508553    3.286126  -22.857143   
2022-07-31   11.733491  -23.124859   -0.262308    7.607341   13.580247   
2022-08-07   -7.115789  -17.171369  -27.436359  -14.154543  -47.272727   
2022-08-14   13.659911  -14.445218  -25.857605   -6.456942    9.836066   
2022-08-21 -116.381514 -188.179811  -49.203283 -100.593759  -17.307692   

                                                                        
State               VT          WA          WI          WV          WY  
DATE                                                                    
2020-03-01         NaN         NaN         NaN         NaN         NaN  
2020-03-08         NaN   48.800000  100.000000         NaN         NaN  
2020-03-15  100.000000   55.516014   97.949886  100.000000         NaN  
2020-03-22   69.841270   68.355856   51.113586   89.795918  100.000000  
2020-03-29   50.263158  -11.417817   25.166667   54.205607        -inf  
...                ...         ...         ...         ...         ...  
2022-07-24   -5.088496  -25.303545   -0.153978   -2.741091  -37.441498  
2022-07-31   13.740458  -13.391464   -7.990762    5.487518   13.495277  
2022-08-07  -10.782241  -11.938765   -6.692293   -1.676245

## Joining with other DataFrames
For the most part, it seems that new cases seem to decline drastically towards the end of our dataset. Let's examine some reasons why that might be the case! If we look at the `JHU_VACCINES` table, we can see the number of doses administered per day in each state. Let's take a look at the week-over-week difference in doses - following the same steps as we did above!

In [21]:
start = perf_counter()
snow_df_vax = pd.read_snowflake(f"(SELECT STABBR AS STATE, DATE, PEOPLE_TOTAL FROM {DATA_PATH}.JHU_VACCINES) UNION ALL (SELECT * FROM VALUES('AK',  DATE('2020-03-01'), NULL))")
snow_df_vax = snow_df_vax.pivot_table(columns=["STATE"], index="DATE", values="PEOPLE_TOTAL")
snow_df_vax = snow_df_vax.resample('7D').sum()
snow_df_vax_diff = snow_df_vax.diff()
repr(snow_df_vax_diff)
end = perf_counter()
print(f"Week-over-week difference of vaccinated peoples took {end-start} seconds end-to-end in Snowpark pandas!")
snow_df_vax_diff

Week-over-week difference of vaccinated peoples took 6.790582207962871 seconds end-to-end in Snowpark pandas!


STATE,AK,AL,AR,AS,AZ,CA,CO,CT,DC,DE,...,TX,UT,VA,VI,VT,WA,WI,WV,WY,NULL
DATE,,,,,,,,,,,,,,,,,,,,,
2020-12-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-21,59718.0,20724.0,16000.0,0.0,0.0,0.0,329150.0,46399.0,0.0,38353.0,...,663885.0,87298.0,188484.0,1300.0,36329.0,260965.0,212389.0,127495.0,16193.0,0.00
2020-12-28,27245.0,106211.0,0.0,0.0,0.0,0.0,273583.0,273409.0,0.0,38784.0,...,1183320.0,117806.0,267397.0,520.0,30606.0,358895.0,105613.0,165246.0,27740.0,0.00
2021-01-04,52079.0,0.0,0.0,0.0,0.0,0.0,279416.0,252318.0,0.0,9976.0,...,1747573.0,213316.0,414409.0,6580.0,49044.0,139349.0,0.0,56051.0,21350.0,0.00
2021-01-11,48325.0,0.0,0.0,0.0,1173629.0,0.0,360568.0,390501.0,0.0,0.0,...,1758700.0,109542.0,497258.0,0.0,39737.0,0.0,407446.0,281272.0,43123.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-22,4502.0,20024.0,16887.0,232.0,62081.0,277070.0,40207.0,29124.0,9198.0,7097.0,...,244274.0,40076.0,44771.0,194.0,6418.0,56411.0,32776.0,0.0,3190.0,29877.75
2022-08-29,5426.0,17436.0,14775.0,300.0,47626.0,239733.0,37366.0,29763.0,0.0,6821.0,...,102507.0,23244.0,89789.0,174.0,5293.0,56134.0,32150.0,0.0,2289.0,28934.25
2022-09-05,1861.0,13721.0,10701.0,338.0,37402.0,170841.0,29796.0,21235.0,0.0,5075.0,...,186100.0,13071.0,66591.0,2459.0,4694.0,43248.0,25646.0,0.0,1726.0,20509.50


In [22]:
# Vanilla pandas
start = perf_counter()


sql = f"SELECT STABBR AS STATE, DATE, PEOPLE_TOTAL FROM {DATA_PATH}.JHU_VACCINES"
cur.execute(sql)

# Fetch the result set from the cursor and deliver it as the pandas DataFrame.
native_df_vax = cur.fetch_pandas_all()

native_df_vax = native_df_vax.pivot_table(columns=["STATE"], index="DATE", values="PEOPLE_TOTAL")
native_df_vax.index = native_pd.to_datetime(native_df_vax.index)
native_df_vax = native_df_vax.resample('7D').sum()
native_df_vax_diff = native_df_vax.diff()
repr(native_df_vax_diff)
end = perf_counter()
print(f"Week-over-week difference of vaccinated peoples took {end-start} seconds end-to-end in vanilla pandas!")
native_df_vax_diff

Week-over-week difference of vaccinated peoples took 0.3088554999558255 seconds end-to-end in vanilla pandas!


STATE,AK,AL,AR,AS,AZ,CA,CO,CT,DC,DE,...,TN,TX,UT,VA,VI,VT,WA,WI,WV,WY
DATE,,,,,,,,,,,,,,,,,,,,,
2020-12-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-21,59718.0,20724.0,16000.0,0.0,0.0,0.0,329150.0,46399.0,0.0,38353.0,...,316369.0,663885.0,87298.0,188484.0,1300.0,36329.0,260965.0,212389.0,127495.0,16193.0
2020-12-28,27245.0,106211.0,0.0,0.0,0.0,0.0,273583.0,273409.0,0.0,38784.0,...,325251.0,1183320.0,117806.0,267397.0,520.0,30606.0,358895.0,105613.0,165246.0,27740.0
2021-01-04,52079.0,0.0,0.0,0.0,0.0,0.0,279416.0,252318.0,0.0,9976.0,...,437254.0,1747573.0,213316.0,414409.0,6580.0,49044.0,139349.0,0.0,56051.0,21350.0
2021-01-11,48325.0,0.0,0.0,0.0,1173629.0,0.0,360568.0,390501.0,0.0,0.0,...,579716.0,1758700.0,109542.0,497258.0,0.0,39737.0,0.0,407446.0,281272.0,43123.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-22,4502.0,20024.0,16887.0,232.0,62081.0,277070.0,40207.0,29124.0,9198.0,7097.0,...,33339.0,244274.0,40076.0,44771.0,194.0,6418.0,56411.0,32776.0,0.0,3190.0
2022-08-29,5426.0,17436.0,14775.0,300.0,47626.0,239733.0,37366.0,29763.0,0.0,6821.0,...,29086.0,102507.0,23244.0,89789.0,174.0,5293.0,56134.0,32150.0,0.0,2289.0
2022-09-05,1861.0,13721.0,10701.0,338.0,37402.0,170841.0,29796.0,21235.0,0.0,5075.0,...,19200.0,186100.0,13071.0,66591.0,2459.0,4694.0,43248.0,25646.0,0.0,1726.0


## Examining the Transformed Data
We see that the number of new people getting vaccinated seems to increase at first, and then begin to decline - which also makes intuitive sense - as we imagine that at first, many people are able to get vaccinated, but as the penetration of the vaccine begins to increase, there are fewer "new" people to vaccinate.

This; however, does not provide us with any insight into why COVID cases might be decreasing. To understand that better, it may be more helpful instead to look at the total number of vaccines administered over time, and compare that to the number of new cases over time. We can accomplish this by joining our `JHU_VACCINES` table, with the week-over-week difference for new cases we computed previously! One issue; however, is that the indexes are mis-aligned. When we resampled weekly, we observe that the two DataFrame's have different starting days, if we try to join the two, we see that we end up with an empty DataFrame:

In [23]:
start = perf_counter()
snow_df_diff.columns = [col[1] for col in snow_df_diff.columns]
snow_df = snow_df_diff.merge(snow_df_vax, left_index=True, right_index=True, suffixes=('_new_cases', '_total_vaccines_administered'))
repr(snow_df)
end = perf_counter()
print(f"Merging on index took {end-start} seconds in Snowpark pandas!")
snow_df

Merging on index took 5.3152353330515325 seconds in Snowpark pandas!


,AK_new_cases,AL_new_cases,AR_new_cases,AZ_new_cases,CA_new_cases,CO_new_cases,CT_new_cases,DC_new_cases,DE_new_cases,FL_new_cases,...,TX_total_vaccines_administered,UT_total_vaccines_administered,VA_total_vaccines_administered,VI_total_vaccines_administered,VT_total_vaccines_administered,WA_total_vaccines_administered,WI_total_vaccines_administered,WV_total_vaccines_administered,WY_total_vaccines_administered,NULL
DATE,,,,,,,,,,,,,,,,,,,,,


In [24]:
start = perf_counter()
native_df_diff.columns = [col[1] for col in native_df_diff.columns]
native_df = native_df_diff.merge(native_df_vax, left_index=True, right_index=True, suffixes=('_new_cases', '_total_vaccines_administered'))
repr(native_df)
end = perf_counter()
print(f"Merging on index took {end-start} seconds in Snowpark pandas!")
native_df

Merging on index took 0.0033911250066012144 seconds in Snowpark pandas!


,AK_new_cases,AL_new_cases,AR_new_cases,AZ_new_cases,CA_new_cases,CO_new_cases,CT_new_cases,DC_new_cases,DE_new_cases,FL_new_cases,...,TN_total_vaccines_administered,TX_total_vaccines_administered,UT_total_vaccines_administered,VA_total_vaccines_administered,VI_total_vaccines_administered,VT_total_vaccines_administered,WA_total_vaccines_administered,WI_total_vaccines_administered,WV_total_vaccines_administered,WY_total_vaccines_administered
DATE,,,,,,,,,,,,,,,,,,,,,


We can resolve this; however, by using the non-resampled data! Let's load back the original pivoted data for both DataFrames, merge them, and then do a resample to look at weekly trends!

In [25]:
# Snowpark pandas
start = perf_counter()
# Read CDC_TESTING
snow_df = pd.read_snowflake(f"{DATA_PATH}.CDC_TESTING")
snow_df = snow_df.rename(columns={"ISO3166_1": "Nation", "ISO3166_2": "State"})
assert (snow_df.Nation == "US").all()
snow_df = snow_df.drop(columns="Nation")
snow_df = snow_df.pivot_table(columns=["State"], values = "POSITIVE", index="DATE")
# READ JHU_VACCINES
snow_df_vax = pd.read_snowflake(f"SELECT STABBR AS STATE, DATE, PEOPLE_TOTAL FROM {DATA_PATH}.JHU_VACCINES")
snow_df_vax = snow_df_vax.pivot_table(columns=["STATE"], index="DATE", values="PEOPLE_TOTAL")
# Merge both and take diff
merged_snow_df = snow_df.merge(snow_df_vax, left_index=True, right_index=True, suffixes=('_new_cases', '_total_vaccines_administered'))
merged_snow_df = merged_snow_df.resample('7D').sum().diff()
repr(merged_snow_df)
end = perf_counter()
print(f"Merging and finding week-over-week difference took {end-start} seconds end-to-end in Snowpark pandas!")
merged_snow_df

Merging and finding week-over-week difference took 12.838606957928278 seconds end-to-end in Snowpark pandas!


State,AK_new_cases,AL_new_cases,AR_new_cases,AZ_new_cases,CA_new_cases,CO_new_cases,CT_new_cases,DC_new_cases,DE_new_cases,FL_new_cases,...,TX_total_vaccines_administered,UT_total_vaccines_administered,VA_total_vaccines_administered,VI_total_vaccines_administered,VT_total_vaccines_administered,WA_total_vaccines_administered,WI_total_vaccines_administered,WV_total_vaccines_administered,WY_total_vaccines_administered,NULL
DATE,,,,,,,,,,,,,,,,,,,,,
2020-12-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-21,-778.0,-6042.0,-2135.0,343.0,-16307.0,-6632.0,-8683.0,7.0,-328.0,-4546.0,...,663885.0,87298.0,188484.0,1300.0,36329.0,260965.0,212389.0,127495.0,16193.0,0.00
2020-12-28,600.0,4468.0,1299.0,7537.0,36305.0,984.0,-1216.0,11.0,-70.0,22086.0,...,1183320.0,117806.0,267397.0,520.0,30606.0,358895.0,105613.0,165246.0,27740.0,0.00
2021-01-04,310.0,-852.0,2308.0,7495.0,-13808.0,2237.0,16961.0,721.0,1160.0,19081.0,...,1747573.0,213316.0,414409.0,6580.0,49044.0,139349.0,0.0,56051.0,21350.0,0.00
2021-01-11,-583.0,-4849.0,-2937.0,-11320.0,-49235.0,-4637.0,-5746.0,-307.0,-560.0,-13356.0,...,1758700.0,109542.0,497258.0,0.0,39737.0,0.0,407446.0,281272.0,43123.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-25,-294.0,-706.0,-865.0,-2504.0,-19161.0,-1427.0,-567.0,-211.0,-64.0,-4641.0,...,352443.0,20126.0,125870.0,469.0,8449.0,74655.0,42522.0,0.0,3608.0,110537.75
2022-08-01,-450.0,-203.0,-336.0,295.0,-24675.0,-1447.0,-123.0,-790.0,142.0,-10939.0,...,-1036877.0,16427.0,79677.0,425.0,6507.0,66434.0,37655.0,0.0,1543.0,36232.00
2022-08-08,-381.0,-561.0,-341.0,-2169.0,-15549.0,-1368.0,-178.0,-319.0,-294.0,-9811.0,...,-643463.0,12762.0,67642.0,419.0,2556.0,61469.0,34944.0,0.0,874.0,39281.75


In [26]:
# Vanilla pandas
start = perf_counter()
# Read CDC_TESTING
sql = f"SELECT * FROM {DATA_PATH}.CDC_TESTING"
cur.execute(sql)
# Fetch the result set from the cursor and deliver it as the pandas DataFrame.
native_df = cur.fetch_pandas_all()
native_df = native_df.rename(columns={"ISO3166_1": "Nation", "ISO3166_2": "State"})
assert (native_df.Nation == "US").all()
native_df = native_df.drop(columns="Nation")
native_df = native_df.pivot_table(columns=["State"], values = "POSITIVE", index="DATE")
# READ JHU_VACCINES
sql = f"SELECT STABBR AS STATE, DATE, PEOPLE_TOTAL FROM {DATA_PATH}.JHU_VACCINES"
cur.execute(sql)
# Fetch the result set from the cursor and deliver it as the pandas DataFrame.
native_df_vax = cur.fetch_pandas_all()
native_df_vax = native_df_vax.pivot_table(columns=["STATE"], index="DATE", values="PEOPLE_TOTAL")
# Merge both and take diff
merged_native_df = native_df.merge(native_df_vax, left_index=True, right_index=True, suffixes=('_new_cases', '_total_vaccines_administered'))
merged_native_df = merged_native_df.resample('7D').sum().diff()
repr(merged_native_df)
end = perf_counter()
print(f"Merging and finding week-over-week difference took {end-start} seconds end-to-end in vanilla pandas!")
merged_native_df

Merging and finding week-over-week difference took 0.8418219169834629 seconds end-to-end in vanilla pandas!


,AK_new_cases,AL_new_cases,AR_new_cases,AZ_new_cases,CA_new_cases,CO_new_cases,CT_new_cases,DC_new_cases,DE_new_cases,FL_new_cases,...,TN_total_vaccines_administered,TX_total_vaccines_administered,UT_total_vaccines_administered,VA_total_vaccines_administered,VI_total_vaccines_administered,VT_total_vaccines_administered,WA_total_vaccines_administered,WI_total_vaccines_administered,WV_total_vaccines_administered,WY_total_vaccines_administered
DATE,,,,,,,,,,,,,,,,,,,,,
2020-12-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-21,-778.0,-6042.0,-2135.0,343.0,-16307.0,-6632.0,-8683.0,7.0,-328.0,-4546.0,...,316369.0,663885.0,87298.0,188484.0,1300.0,36329.0,260965.0,212389.0,127495.0,16193.0
2020-12-28,600.0,4468.0,1299.0,7537.0,36305.0,984.0,-1216.0,11.0,-70.0,22086.0,...,325251.0,1183320.0,117806.0,267397.0,520.0,30606.0,358895.0,105613.0,165246.0,27740.0
2021-01-04,310.0,-852.0,2308.0,7495.0,-13808.0,2237.0,16961.0,721.0,1160.0,19081.0,...,437254.0,1747573.0,213316.0,414409.0,6580.0,49044.0,139349.0,0.0,56051.0,21350.0
2021-01-11,-583.0,-4849.0,-2937.0,-11320.0,-49235.0,-4637.0,-5746.0,-307.0,-560.0,-13356.0,...,579716.0,1758700.0,109542.0,497258.0,0.0,39737.0,0.0,407446.0,281272.0,43123.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-25,-294.0,-706.0,-865.0,-2504.0,-19161.0,-1427.0,-567.0,-211.0,-64.0,-4641.0,...,41093.0,352443.0,20126.0,125870.0,469.0,8449.0,74655.0,42522.0,0.0,3608.0
2022-08-01,-450.0,-203.0,-336.0,295.0,-24675.0,-1447.0,-123.0,-790.0,142.0,-10939.0,...,37387.0,-1036877.0,16427.0,79677.0,425.0,6507.0,66434.0,37655.0,0.0,1543.0
2022-08-08,-381.0,-561.0,-341.0,-2169.0,-15549.0,-1368.0,-178.0,-319.0,-294.0,-9811.0,...,36993.0,-643463.0,12762.0,67642.0,419.0,2556.0,61469.0,34944.0,0.0,874.0


# Integration with Snowpark ML
Now that we have the week-over-week trends in new cases and new vaccine administrations, we can attempt to see if our intuition - that more new vaccines administered leads to fewer new cases - holds out. One common way to do this is to attempt to fit a linear regression on the data, and compute the $r^2$ value for the predictions generated by that linear regression. To keep things simple, let's focus on a single state - say Alaska, and attempt to compute the linear regression between new cases and new vaccines administered. There is no native API in pandas to compute the linear regression between two Series; however, Snowflake's ML API for Python's `LinearRegression` model does exactly what we need. Because our data never left Snowflake, we could use this function to compute the value we need; however, the model requires a Snowpark DataFrame as an input, and we are working with a Snowpark pandas DataFrame.

Fortunately, Snowpark pandas integrates with Snowpark (which can be used to call the SQL function) with the `to_snowpark` and `to_snowpark_pandas` methods. We can use these methods to get a Snowpark DataFrame and compute linear regression!

In [27]:
start = perf_counter()
snowpark_df = merged_snow_df.dropna().to_snowpark()
end = perf_counter()
print(f"Took {end - start} seconds to drop NA values and convert to a Snowpark DataFrame")

Took 0.6368929999880493 seconds to drop NA values and convert to a Snowpark DataFrame


In [28]:
from snowflake.ml.modeling.linear_model import LinearRegression

In [29]:
lin_reg_model = LinearRegression(input_cols='"AK_total_vaccines_administered"', label_cols='"AK_new_cases"')
lin_reg_model = lin_reg_model.fit(snowpark_df)
lin_reg_model.score(snowpark_df)

0.004365500752932738

Ultimately, it seems that our intuition was wrong - an $r^2$ value of 0.004 is quite low, and seems to indicate that there may not be a linear relationship between the number of total vaccines administered per week and the number of new cases per week. Epidemiological data is quite complex, and so it makes sense that the model we built based off of our assumptions ended up not having the modeling capacity necessary for this problem - perhaps a more detailed deep dive into the problem would yield a better model!

Hopefully, though, the takeaway from this notebook will be that Snowpark pandas is a powerful tool that can help you combine the expressiveness of pandas with the scale and compute capabilities of the Snowflake ecosystem!